# DSA210 Project – Apple vs Samsung Stock Analysis (2019–2024)

This notebook analyzes how the COVID-19 pandemic influenced the stock price behavior 
of two major technology companies: **Apple** and **Samsung**. By combining daily stock prices 
with global daily COVID-19 case counts, this study investigates whether external global shocks 
can affect investor behavior, market volatility, and company growth patterns.

This submission corresponds to the **28 November milestone**, which includes:
- Data collection  
- Data cleaning and preprocessing  
- Exploratory data analysis (EDA)  
- Statistical hypothesis testing  


## Research Questions

- How did Apple and Samsung stock prices change between 2019 and 2024?
- Did both companies show similar growth patterns before and after the COVID-19 pandemic?
- Is there a relationship between daily COVID-19 cases and the stock prices of Apple and Samsung?
- How can the average prices and returns of Apple and Samsung be compared over time?
- Which company adapted more quickly to the market changes caused by the pandemic?

## Hypotheses

- **H1:** In the early pandemic months, Apple's stock price increased faster than Samsung's.
- **H2:** Samsung reacts more slowly to market shocks, lagging behind Apple's price growth.
- **H3:** Global COVID-19 daily case counts are positively correlated with Apple’s stock price.
- **H4:** After 2021, Samsung exhibits higher volatility compared to Apple.
- **H5:** Apple outperforms Samsung in terms of monthly average returns.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, pearsonr

plt.rcParams["figure.figsize"] = (12, 4)
sns.set(style="whitegrid")

## 1. Data Collection

In this section, Apple, Samsung, and COVID-19 datasets are loaded and merged into a single
dataframe using the `Date` column.


In [ ]:
# Load CSV datasets
apple = pd.read_csv("apple.csv")
samsung = pd.read_csv("samsung.csv")
covid = pd.read_csv("corona.csv")

# Automatically detect Samsung price column
possible_price_cols = ["Close", "Price", "Adj Close", "Adj_Close", "Closing Price"]
samsung_price_col = None

for col in possible_price_cols:
    if col in samsung.columns:
        samsung_price_col = col
        break

print("Samsung price column found:", samsung_price_col)

In [ ]:
# Fix date formats
apple["Date"] = pd.to_datetime(apple["Date"], format="mixed", dayfirst=True)
samsung["Date"] = pd.to_datetime(samsung["Date"], format="mixed", dayfirst=True)
covid["Date"] = pd.to_datetime(covid["Date"], format="mixed", dayfirst=True)

# Merge datasets
df = apple.merge(samsung, on="Date", how="inner")
df = df.merge(covid, on="Date", how="left")

# Rename columns consistently
df = df.rename(columns={
    "Close": "Apple_Price",
    samsung_price_col: "Samsung_Price",
    "New cases": "Covid_Cases"
})

df.head()

## 2. Data Cleaning & Preparation

The following cleaning steps are applied:
- Convert price and COVID-19 case values into numeric format  
- Remove commas from COVID case values  
- Prepare variables for analysis  


In [ ]:
# Clean numeric columns
for col in ["Apple_Price", "Samsung_Price", "Covid_Cases"]:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

df.describe()

## 3. Exploratory Data Analysis (EDA)

This section includes:
- Stock price trends  
- COVID-19 global case trends  
- Correlation analysis  
- Daily return distributions  


In [ ]:
# Apple vs Samsung stock prices
plt.plot(df["Date"], df["Apple_Price"], label="Apple")
plt.plot(df["Date"], df["Samsung_Price"], label="Samsung")
plt.title("Apple vs Samsung Stock Prices (2019–2024)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()

# COVID-19 daily cases
covid_part = df.dropna(subset=["Covid_Cases"])
plt.plot(covid_part["Date"], covid_part["Covid_Cases"], alpha=0.5)
plt.title("Daily COVID-19 Cases (Global, 2020)")
plt.xlabel("Date")
plt.ylabel("New Cases")
plt.show()

In [ ]:
# Correlation matrix
corr_mat = df[["Apple_Price", "Samsung_Price", "Covid_Cases"]].corr()
sns.heatmap(corr_mat, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation Matrix")
plt.show()

corr_mat

In [ ]:
# Daily returns
df["Apple_Return"] = df["Apple_Price"].pct_change()
df["Samsung_Return"] = df["Samsung_Price"].pct_change()

sns.histplot(df["Apple_Return"], kde=True, label="Apple")
sns.histplot(df["Samsung_Return"], kde=True, label="Samsung")
plt.legend()
plt.title("Daily Return Distribution")
plt.show()

## 4. Hypothesis Testing

This section tests hypotheses H1–H5 using:
- Independent t-tests  
- Pearson correlations  
- Volatility (standard deviation of returns)  


In [ ]:
# H1: Early COVID period growth comparison
early_period = df[df["Date"] < "2020-06-01"]

apple_growth = early_period["Apple_Return"].dropna()
samsung_growth = early_period["Samsung_Return"].dropna()

t_stat, p_val = ttest_ind(apple_growth, samsung_growth)
print("H1 Apple vs Samsung Early Growth T-test")
print("t =", t_stat, " p =", p_val)

In [ ]:
# H2 & H5: Monthly average prices
df["Month"] = df["Date"].dt.to_period("M")
monthly = df.groupby("Month")[["Apple_Price", "Samsung_Price"]].mean().reset_index()

t_stat2, p_val2 = ttest_ind(monthly["Apple_Price"], monthly["Samsung_Price"])
print("Monthly Price Comparison T-test")
print("t =", t_stat2, " p =", p_val2)

In [ ]:
# H3: Correlation between COVID-19 cases and stock prices
apple_corr_df = df[["Covid_Cases", "Apple_Price"]].dropna()
samsung_corr_df = df[["Covid_Cases", "Samsung_Price"]].dropna()

corr_apple, p_apple = pearsonr(apple_corr_df["Covid_Cases"], apple_corr_df["Apple_Price"])
corr_samsung, p_samsung = pearsonr(samsung_corr_df["Covid_Cases"], samsung_corr_df["Samsung_Price"])

print("COVID–Apple Price Correlation:", corr_apple, "p =", p_apple)
print("COVID–Samsung Price Correlation:", corr_samsung, "p =", p_samsung)

In [ ]:
# H4: Post-2021 volatility comparison
post21 = df[df["Date"] > "2021-01-01"]

vol_apple = post21["Apple_Return"].std()
vol_samsung = post21["Samsung_Return"].std()

print("Apple Volatility (post-2021):", vol_apple)
print("Samsung Volatility (post-2021):", vol_samsung)

## 5. Summary of Findings (28 November Stage)

- Apple and Samsung stock prices show a strong positive correlation (≈ 0.71).
- Apple stock prices are strongly correlated with global COVID-19 cases, 
  while Samsung shows no meaningful correlation.
- **H1 rejected**: early-pandemic growth of Apple and Samsung is not significantly different.
- **H2 rejected** and **H5 supported**: Samsung has higher prices, but Apple demonstrates more stable returns.
- **H4 accepted**: Samsung is about 3× more volatile than Apple after 2021.

This concludes the data collection, cleaning, EDA, and hypothesis testing phase.
The next milestone (02 January) will involve machine learning models for prediction and comparison.
